<a href="https://colab.research.google.com/github/essiesalari/Australian-Celebrities-Face-Recognition-with-Pretrained_Models/blob/main/Australian_Celebrities_Face_Recognition_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Australian Celebrities Face Recognition_Pretrained Models**

## Import Libraries

In [23]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Import datatset

In [24]:
! pip install bing_image_downloader

In [25]:
from bing_image_downloader import downloader

In [26]:
# Query
search_queries=[
    'Nicole Kidman',
    'Liam Hemsworth',
    'Cathy Freeman',
    'Margot Robbie',
    'Mel Gibson'
]

dataset_path = 'dataset'

# Imgae downloder function
def donwloader(query, n_sample, dataset_name):
  downloader.download(query, limit=n_sample, output_dir=os.path.join(dataset_path, dataset_name))

for query in search_queries:
  donwloader(query, n_sample=30, dataset_name='train')
  donwloader(query, n_sample=5, dataset_name='val')
  donwloader(query, n_sample=3, dataset_name='test')

[%] Downloading Images to /content/dataset/train/Nicole Kidman


[!!]Indexing page: 1

[%] Indexed 35 Images on Page 1.


[%] Downloading Image #1 from https://www.hawtcelebs.com/wp-content/uploads/2012/05/NICOLE-KIDMAN-at-The-Paperboy-Premiere-at-65th-Annual-Cannes-Film-Festival-13.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://celebmafia.com/wp-content/uploads/2016/04/nicole-kidman-the-family-fang-premiere-in-new-york-city-3.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://celebmafia.com/wp-content/uploads/2018/11/nicole-kidman-photoshoot-for-usa-today-november-2018-3.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://celebmafia.com/wp-content/uploads/2016/04/nicole-kidman-academy-of-country-music-awards-2016-in-las-vegas-1.jpg
[%] File Downloaded !

[%] Downloading Image #5 from http://images5.fanpop.com/image/photos/31800000/Nicole-Kidman-nicole-kidman-31862066-1440-2200.jpg
[%] File Downloaded !

[%] Downloading Image #6 from https:/

In [27]:
dataset_path='/content/dataset'
train_path = os.path.join(dataset_path, 'train')
val_path = os.path.join(dataset_path, 'val')
test_path = os.path.join(dataset_path, 'test')

In [28]:
nr_files = nr_train_files = nr_val_files = nr_test_files = 0

for root, dirc, files in os.walk(dataset_path):
  nr_files += len(files)
print("#files: ", nr_files)
for root, dirc, files in os.walk(train_path):
  nr_train_files += len(files)
print("#train_files: ", nr_train_files)
for root, dirc, files in os.walk(val_path):
  nr_val_files += len(files)
print("#val_files: ", nr_val_files)
for root, dirc, files in os.walk(test_path):
  nr_test_files += len(files)
print("#test_files: ", nr_test_files)

#files:  248
#train_files:  208
#val_files:  25
#test_files:  15


# Build  a New Model

In [29]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3, EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [56]:
batch_size = 128
lr = 0.001
epochs = 100

# Define Image data generator
train_datagen = ImageDataGenerator(
                                  rescale=1./255,
                                  rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen =  ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

# Print the number of images belonging to each class in the training set
class_counts = np.bincount(train_generator.classes)
class_labels = list(train_generator.class_indices.keys())
for label, count in zip(class_labels, class_counts):
    print(f"The class {label} has {count} images in the training set.")

Found 208 images belonging to 5 classes.
Found 25 images belonging to 5 classes.
Found 15 images belonging to 5 classes.
The class Cathy Freeman has 48 images in the training set.
The class Liam Hemsworth has 32 images in the training set.
The class Margot Robbie has 32 images in the training set.
The class Mel Gibson has 60 images in the training set.
The class Nicole Kidman has 36 images in the training set.


## VGG16

In [59]:
# Load VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Fine-tunig
for layer in base_model.layers[:-10]:
  layer.trainable = False

# Add global average layer and dropout for regularization
net = base_model.output
net = GlobalAveragePooling2D()(net)
net = Dropout(0.6)(net)

# Creat the new model with VGG16 (feature selection) + SVC (classifier)
new_model = Model(inputs=base_model.input, outputs=net)

train_features = new_model.predict(train_generator)
train_labels = train_generator.classes

val_features = new_model.predict(val_generator)
val_labels = val_generator.classes

# Train SVM Classifier
svm = SVC(kernel='linear')
svm.fit(train_features, train_labels )

# Evaluate on validation set
print("Validation Accuracy:", svm.score(val_features, val_labels))

1/1 [==============================] - 0s 399ms/step
Validation Accuracy: 0.2


## ResNet50

In [49]:
# Load Resnet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Fine-tunig
for layer in base_model.layers[:-10]:
  layer.trainable = False

# Add global average layer and dropout for regularization
net = base_model.output
net = GlobalAveragePooling2D()(net)
net = Dropout(0.6)(net)

# Creat the new model with VGG16 (feature selection) + SVC (classifier)
new_model = Model(inputs=base_model.input, outputs=net)

train_features = new_model.predict(train_generator)
train_labels = train_generator.classes

val_features = new_model.predict(val_generator)
val_labels = val_generator.classes

# Train SVM Classifier
svm = SVC(kernel='linear')
svm.fit(train_features, train_labels )

# Evaluate on validation set
print("Validation Accuracy:", svm.score(val_features, val_labels))

1/1 [==============================] - 0s 413ms/step
Validation Accuracy: 0.16


## InceptionV3

In [50]:
# Load Resnet50
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Fine-tunig
for layer in base_model.layers[:-4]:
  layer.trainable = False

# Add global average layer and dropout for regularization
net = base_model.output
net = GlobalAveragePooling2D()(net)
net = Dropout(0.6)(net)

# Creat the new model with VGG16 (feature selection) + SVC (classifier)
new_model = Model(inputs=base_model.input, outputs=net)

train_features = new_model.predict(train_generator)
train_labels = train_generator.classes

val_features = new_model.predict(val_generator)
val_labels = val_generator.classes

# Train SVM Classifier
svm = SVC(kernel='linear')
svm.fit(train_features, train_labels )

# Evaluate on validation set
print("Validation Accuracy:", svm.score(val_features, val_labels))

1/1 [==============================] - 0s 422ms/step
Validation Accuracy: 0.28


## EfficientNetB0

In [51]:
# Load Resnet50
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Fine-tunig
for layer in base_model.layers[:-10]:
  layer.trainable = False

# Add global average layer and dropout for regularization
net = base_model.output
net = GlobalAveragePooling2D()(net)
net = Dropout(0.6)(net)

# Creat the new model with VGG16 (feature selection) + SVC (classifier)
new_model = Model(inputs=base_model.input, outputs=net)

train_features = new_model.predict(train_generator)
train_labels = train_generator.classes

val_features = new_model.predict(val_generator)
val_labels = val_generator.classes

# Train SVM Classifier
svm = SVC(kernel='linear')
svm.fit(train_features, train_labels )

# Evaluate on validation set
print("Validation Accuracy:", svm.score(val_features, val_labels))

1/1 [==============================] - 1s 562ms/step
Validation Accuracy: 0.2
